In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import keras
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from keras.applications import Xception
from keras.applications.xception import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

2023-09-27 16:52:29.050066: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-27 16:52:29.054251: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 16:52:29.145167: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 16:52:29.147027: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 16:52:29.705160: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

# Récupérer les listes images et texte dans un fichier texte

In [2]:
import pandas as pd

#df = pd.read_csv('/mnt/c/Users/steph/OneDrive/Images/Rakuten/df_cleaned.csv')
df = pd.read_csv('/mnt/c/Users/steph/OneDrive/Images/Rakuten/X_train_bis.csv')

# Création d'un dictionnaire pour stocker les listes d'image_name, designation et description par label
label_image_info = {}

# Grouper les données par le label (prdtypecode)
grouped = df.groupby('prdtypecode')

# Parcourir chaque groupe et collecter les 'image_name', designation et description
for label, group in grouped:
    image_info = {
        'image_name': group['image_name'].tolist(),
        'designation': group['designation'].tolist(),
        'description': group['description'].tolist()
    }
    label_image_info[label] = image_info

# Écrire les listes dans un fichier texte
with open('label_txt_img_info.txt', 'w', encoding='utf-8') as file:
    for label, image_info in label_image_info.items():
        file.write(f"Label {label}:\n")
        for i in range(len(image_info['image_name'])):
            file.write(f"Image Name: {image_info['image_name'][i]}\n")
            file.write(f"Designation: {image_info['designation'][i]}\n")
            file.write(f"Description: {image_info['description'][i]}\n\n")

print("Les listes ont été écrites dans le fichier label_txt_img_info.txt.")



Les listes ont été écrites dans le fichier label_txt_img_info.txt.


# Texte

In [3]:
stop_words = stopwords.words('french') + stopwords.words('english') + stopwords.words('german') +[ \
                'plus', 'peut', 'tout', 'etre', 'sans', 'dont', 'aussi',  \
                  'comme', 'meme', 'bien','leurs', 'elles', 'cette','celui',   \
                  'ainsi', 'encore', 'alors', 'toujours', 'toute','deux', 'nouveau',   \
                  'peu', 'car', 'autre', 'jusqu', 'quand', 'ici', 'ceux', 'enfin',  \
                  'jamais', 'autant', 'tant', 'avoir', 'moin', 'celle', 'tous',   \
                  'contre', 'pourtant', 'quelque', 'toutes', 'surtout', 'cet',  \
                  'comment', 'rien', 'avant', 'doit', 'autre', 'depuis', 'moins',  \
                  'tre', 'souvent', 'etait', 'pouvoir', 'apre', 'non', 'ver', 'quel',   \
                  'pourquoi', 'certain', 'fait', 'faire', 'sou', 'donc', 'trop',  \
                  'quelques', 'parfois', 'tres', 'donc', 'dire', 'eacute', 'egrave',  \
                  'rsquo', 'agrave', 'ecirc', 'nbsp', 'acirc', 'apres', 'autres',  \
                  'ocirc', 'entre', 'sous', 'quelle']

In [4]:
def CreateTextANDcleaning(data):
    df = data
    #valeurs MANQUANTES
    df['designation'] = df['designation'].astype('string')
    df['description'] = df['description'].astype('string')

    # remplacement des Nan par une chaîne de caractère
    df['description'].fillna('', inplace=True)

    #create text
    df['text']=""
    for i in range(df.shape[0]):
        df['text'][i] = create_text(df['designation'][i], df['description'][i])

    df['text'] = df['text'].str.split()

    df['text'] = df['text'].apply(lambda x: unique_description(x))
    df['text'] = df['text'].apply(lambda x: " ".join(x))

    df['text'] = df['text'].apply(lambda text : lower_case(text))
    df['text'] = df['text'].apply(lambda text : remove_accent(text))

    df['text'] = df['text'].apply(lambda text : remove_htmltags(text))
    df['text'] = df['text'].apply(lambda text : keeping_essentiel(text))

    # Initialiser la variable des mots vides
    df['text'] = df['text'].str.split()
    df['text']= df['text'].apply(lambda x: operation(x))

    df['text'] = df['text'].apply(lambda x: " ".join(x))

    return df['text']


def create_text(text1, text2):
    if pd.isna(text2):
        text = text1
    else:
        text = text1 +" "+ text2
    return text

def unique_description(text):
    unique=[text[0]]
    for mot in text:
        if mot not in unique:
            unique.append(mot)
    return unique

def lower_case(text):
    text = text.lower().strip()
    return text

def remove_accent(string):
    string = string.replace('á', 'a')
    string = string.replace('â', 'a')

    string = string.replace('é', 'e')
    string = string.replace('è', 'e')
    string = string.replace('ê', 'e')
    string = string.replace('ë', 'e')

    string = string.replace('î', 'i')
    string = string.replace('ï', 'i')

    string = string.replace('ö', 'o')
    string = string.replace('ô', 'o')
    string = string.replace('ò', 'o')
    string = string.replace('ó', 'o')

    string = string.replace('ù', 'u')
    string = string.replace('û', 'u')
    string = string.replace('ü', 'u')

    string = string.replace('ç', 'c')

    return string

def remove_htmltags(text):
    text = re.sub('<[^<]+?>', '',text)
    return text

def keeping_essentiel(text):
    text = re.sub(r"[^a-zA-Z]+", " ", text)
    return text

def operation(x):
    my_list=[]
    for mot in x:
        if (mot not in stop_words and len(mot)>2):
            my_list.append(mot)
    return my_list

In [5]:
designation1 = "Un Cyclone A La Jamaique."
description1 = "Traduit de l'anglais par JEAN TALVA."


df = {'designation': [designation1],
        'description': [description1]}

# création d'un dataframe
df_text = pd.DataFrame(df)
df_text = df_text.astype(str)

df_text.head()

,designation,description
0,Un Cyclone A La Jamaique.,Traduit de l'anglais par JEAN TALVA.


In [6]:
# Nettoyage du texte avec la fonction CreateTextANDcleaning
CreateTextANDcleaning(df_text)

print(df_text)

                 designation                           description  \
0  Un Cyclone A La Jamaique.  Traduit de l'anglais par JEAN TALVA.   

                                          text  
0  cyclone jamaique traduit anglais jean talva  


/tmp/ipykernel_213794/1898794411.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = create_text(df['designation'][i], df['description'][i])


In [7]:
prompt = df_text['text'].astype(str)

In [8]:
print(prompt)

0    cyclone jamaique traduit anglais jean talva
Name: text, dtype: object


In [9]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(prompt)

In [10]:
maxlen = 400 
text = tokenizer.texts_to_sequences(prompt)

text = keras.preprocessing.sequence.pad_sequences(text,
                                                  maxlen = maxlen,
                                                  padding='post')    

In [11]:
print(text)

[[1 2 3 4 5 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]


In [12]:
#model_conv1D = load_model('/mnt/c/Users/steph/OneDrive/Images/Rakuten/save_data/model.h5/DL_RNN_Conv1D/model.h5_Conv1D_08092023.h5')

model_conv1D = load_model('/mnt/c/Users/steph/OneDrive/Images/Rakuten/save_data_bis/model_github_save/conv1d_model.h5')
#model_conv1D = load_model('/mnt/c/Users/steph/OneDrive/Images/Rakuten/save_data_bis/Text/weights_model_2309/model_rnn1.h5')

#weights_conv1D = load_weights('weights_conv1d.h5')
#model_conv1D = load_model('model_conv1d.h5')

2023-09-27 16:52:31.205576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 16:52:31.340404: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
# prédiction conv1D sur le texte tokenizé 
y_pred_text = model_conv1D.predict(text)

1/1 [==============================] - 0s 65ms/step


In [14]:
y_pred_text

array([[9.0158703e-03, 1.2708700e-02, 5.4030879e-03, 3.5592949e-04,
        1.9397237e-03, 1.1224814e-03, 1.8702996e-03, 3.3140709e-03,
        6.1604154e-04, 5.4257684e-03, 1.9925074e-03, 1.0485913e-01,
        6.2565743e-03, 4.7089392e-03, 1.0588962e-02, 2.8872348e-02,
        4.8137497e-02, 1.5715798e-02, 3.2164801e-03, 4.2344056e-04,
        6.2436488e-04, 7.5107366e-03, 5.2588056e-03, 6.4597678e-01,
        7.1760580e-02, 2.3161517e-03, 8.8409261e-06]], dtype=float32)

In [15]:
# Classe prédite en utilisant argmax
y_pred_class = np.argmax(y_pred_text, axis=1)

In [16]:
prob_class_text = y_pred_text * 100

print(prob_class_text)

[[9.01587009e-01 1.27086997e+00 5.40308774e-01 3.55929472e-02
  1.93972379e-01 1.12248145e-01 1.87029958e-01 3.31407100e-01
  6.16041534e-02 5.42576849e-01 1.99250743e-01 1.04859133e+01
  6.25657439e-01 4.70893919e-01 1.05889618e+00 2.88723493e+00
  4.81374979e+00 1.57157981e+00 3.21648002e-01 4.23440561e-02
  6.24364875e-02 7.51073658e-01 5.25880575e-01 6.45976791e+01
  7.17605782e+00 2.31615171e-01 8.84092588e-04]]


# Image

In [17]:
image_path = '/mnt/c/Users/steph/OneDrive/Images/Rakuten/images/image_train/image_975548119_product_225089670.jpg'

In [18]:
#weights_xception = load_weights('weights_xception.h5')
model_xception = load_model('/mnt/c/Users/steph/OneDrive/Images/Rakuten/save_data_bis/model_github_save/checkpoint_Xception_model.h5')

In [19]:
img = image.load_img(image_path, target_size=(299, 299))  # Xception : taille d'image de 299x299
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

In [20]:
# prédiction avec modèle xception sur les images 
y_pred_image = model_xception.predict(img)

1/1 [==============================] - 0s 369ms/step


In [21]:
y_pred_image

array([[7.69720376e-01, 1.63446832e-02, 7.89960450e-06, 2.50825597e-05,
        1.01187848e-04, 3.55430529e-05, 3.00065294e-04, 2.63054215e-04,
        9.90819535e-05, 6.01580650e-05, 4.20229360e-02, 1.00397326e-01,
        2.17743254e-05, 5.27036435e-04, 2.93634279e-04, 9.56937147e-05,
        1.51205404e-05, 1.52210059e-05, 2.79477108e-02, 1.06002763e-03,
        4.87955513e-05, 2.92930286e-04, 3.01220780e-03, 3.30895069e-03,
        8.84286070e-04, 3.30458358e-02, 5.33855491e-05]], dtype=float32)

In [22]:
prob_class_img = y_pred_image * 100

print(prob_class_img)

[[7.6972038e+01 1.6344683e+00 7.8996044e-04 2.5082559e-03 1.0118784e-02
  3.5543053e-03 3.0006530e-02 2.6305422e-02 9.9081956e-03 6.0158065e-03
  4.2022934e+00 1.0039733e+01 2.1774326e-03 5.2703645e-02 2.9363427e-02
  9.5693711e-03 1.5120540e-03 1.5221006e-03 2.7947712e+00 1.0600276e-01
  4.8795552e-03 2.9293029e-02 3.0122077e-01 3.3089507e-01 8.8428609e-02
  3.3045835e+00 5.3385547e-03]]


In [23]:
import pandas as pd

# Données
prdtypecode = [10, 40, 50, 60, 1140, 1160, 1180, 1280, 1281,
               1300, 1301, 1302, 1320, 1560, 1920, 1940, 2060,
               2220, 2280, 2403, 2462, 2522, 2582, 2583,
               2585, 2705, 2905]

labels = ["Livres adultes", "Jeux Vidéos", "Accessoires de Jeux Vidéos", "Consoles de jeux", "Figurine", "Carte à Collectionner",
          "Masques", "Jouets pour Enfants", "Jeux de Cartes et de société", "Produits télécommandés",
          "Vêtements pour enfants", "Jouets pour Enfants", "Produits Bébés et Enfants",
          "Literies et Meubles", "Accessoires Maison", "Alimentation", "Décoration d'intérieur", "Accessoires Animaux",
          "Journaux et Magazines", "Livres et Revues", "Jeux", "Papeterie",
          "Mobilier", "Piscine", "Jardinage", "Livres", "Jeux en ligne et Logiciels"]

# fonction reshape pour convertir en tableau 1D
prob_class_text = (y_pred_text.reshape(-1)) * 100
prob_class_img = (y_pred_image.reshape(-1)) * 100

# Création du DataFrame
data = {'code_produit': prdtypecode, 'labels': labels, 'prob_class_text': prob_class_text, 'prob_class_img': prob_class_img}
df = pd.DataFrame(data)

# Affichage du DataFrame
display(df)


,code_produit,labels,prob_class_text,prob_class_img
0,10,Livres adultes,0.901587,76.972038
1,40,Jeux Vidéos,1.270870,1.634468
2,50,Accessoires de Jeux Vidéos,0.540309,0.000790
3,60,Consoles de jeux,0.035593,0.002508
4,1140,Figurine,0.193972,0.010119
5,1160,Carte à Collectionner,0.112248,0.003554
6,1180,Masques,0.187030,0.030007
7,1280,Jouets pour Enfants,0.331407,0.026305
8,1281,Jeux de Cartes et de société,0.061604,0.009908
9,1300,Produits télécommandés,0.542577,0.006016


In [24]:
# Indice de la valeur maximale pour prob_class_text
idx_max_text = df['prob_class_text'].idxmax()

# Indice de la valeur maximale pour prob_class_img
idx_max_img = df['prob_class_img'].idxmax()

# Suppression des colonnes 
df_temp = df.drop(columns=['prob_class_text', 'prob_class_img'])

# Affichez les valeurs maximales (sans afficher la colonne)
print(df_temp.loc[idx_max_text])

# Valeur maximale pour prob_class_img (sans afficher la colonne)
print(df_temp.loc[idx_max_img])


code_produit       2583
labels          Piscine
Name: 23, dtype: object
code_produit                10
labels          Livres adultes
Name: 0, dtype: object


In [25]:
# Indice de la valeur maximale pour prob_class_text
idx_max_text = df['prob_class_text'].idxmax()

# Indice de la valeur maximale pour prob_class_img
idx_max_img = df['prob_class_img'].idxmax()

# Valeur maximale pour prob_class_text
print(df.loc[idx_max_text])

# Valeur maximale pour prob_class_img
print(df.loc[idx_max_img])

code_produit            2583
labels               Piscine
prob_class_text    64.597679
prob_class_img      0.330895
Name: 23, dtype: object
code_produit                   10
labels             Livres adultes
prob_class_text          0.901587
prob_class_img          76.972038
Name: 0, dtype: object


In [26]:
# Poids pour le modèle Conv1D
conv1D_weight = 0.6
# Poids pour le modèle Xception
xception_weight = 0.4


# Calcul des prédictions pondérées pour chaque image
df['weighted_proba'] = (df['prob_class_text'] * conv1D_weight + df['prob_class_img'] * xception_weight) / (conv1D_weight + xception_weight)

# Sélection des classes prédites pour chaque image
df['predicted_classes'] = np.argmax(df[['prob_class_text', 'prob_class_img']].values, axis=1).astype(int)

# Calcul des probabilités en pourcentage pour chaque image
df['class_probabilities'] = df['weighted_proba'] * 100

# Calcul des prédictions fusionnées pour chaque image (calcul de la moyenne pondérée)
df['image_predictions'] = df['prob_class_img'] * xception_weight
df['text_predictions'] = df['prob_class_text'] * conv1D_weight

# Regroupement des prédictions par classe et calcul de la moyenne pondérée
image_grouped = df.groupby('predicted_classes')['image_predictions'].mean()
text_grouped = df.groupby('predicted_classes')['text_predictions'].mean()

# Sélection la classe prédite en fonction de la moyenne pondérée maximale
df['image_predicted_class'] = image_grouped.idxmax()
df['text_predicted_class'] = text_grouped.idxmax()

# Sélection des codes de produit associés aux classes prédites
df['image_predicted_code'] = df['code_produit'].loc[df['image_predicted_class']].values
df['text_predicted_code'] = df['code_produit'].loc[df['text_predicted_class']].values

# Sélection des labels associés aux classes prédites
df['image_predicted_label'] = df['labels'].loc[df['image_predicted_class']].values
df['text_predicted_label'] = df['labels'].loc[df['text_predicted_class']].values

# Affichage des résultats
print("Prédictions fusionnées pour l'image (Xception) :")
print("Classe prédite :", df['image_predicted_class'].values[0])
print("Code de produit associé :", df['image_predicted_code'].values[0])
print("Label associé :", df['image_predicted_label'].values[0])

print("\nPrédictions fusionnées pour le texte (Conv1D) :")
print("Texte à prédire :")
print(df_text['text'].values[0])  # Affichez le texte du premier exemple
print("Classe prédite :", df['text_predicted_class'].values[0])
print("Code de produit associé :", df['text_predicted_code'].values[0])
print("Label associé :", df['text_predicted_label'].values[0])

# Récupération des indices des deux classes les plus probables
top_classes_indices = np.argsort(-df['weighted_proba'].values[0])[:2]
# Pourcentages des deux classes les plus probables
top_class_probabilities = df['class_probabilities'].values[0][top_classes_indices]

# Affichage des deux premières classes prédites avec leurs pourcentages
print("\nDeux premières classes prédites (Xception) :")
for i in range(len(top_classes_indices)):
    class_index = top_classes_indices[i]
    class_label = df['labels'].iloc[class_index]
    class_probability = top_class_probabilities[i]
    print(f"Classe {class_index}: {class_label} - {class_probability:.2f}%")

# Récupération des indices des deux classes les plus probables sur le texte
top_text_classes_indices = np.argsort(-df['text_predictions'].values[0])[:2]
top_text_class_probabilities = df['class_probabilities'].values[0][top_text_classes_indices]

print("\nDeux premières classes prédites (Conv1D) :")
for i in range(len(top_text_classes_indices)):
    class_index = top_text_classes_indices[i]
    class_label = df['labels'].iloc[class_index]
    class_probability = top_text_class_probabilities[i]
    print(f"Classe {class_index}: {class_label} - {class_probability:.2f}%")


Prédictions fusionnées pour l'image (Xception) :
Classe prédite : 1
Code de produit associé : 40
Label associé : Jeux Vidéos

Prédictions fusionnées pour le texte (Conv1D) :
Texte à prédire :
cyclone jamaique traduit anglais jean talva
Classe prédite : 0
Code de produit associé : 10
Label associé : Livres adultes


IndexError: invalid index to scalar variable.